In [1]:
# 16-　コメント
import random

# 親クラスでプレイヤーとモンスターの共通部分を設定する
class Character:
    # 共通で持つ要素（HP、攻撃力、MPなど）を設定する
    def __init__(self, name, HP, attack_power, MP, magic_power):
        self.name   = name
        self.hp     = HP
        self.attack = attack_power
        self.mp = MP
        self.magic = magic_power
        self.max_hp = HP

    # 共通の行動3パターン（物理攻撃・攻撃魔法・回復魔法）を設定する
    # 全員、物理攻撃
    def common_attack(self, attack_target, heal_target, attack_message):
        # 会心・ミス・通常攻撃の3パターンからランダムに物理攻撃を行う
        physical_attack_list =  [  1,   2,   3]
        weights              =  [0.1, 0.2, 0.7]
        physical_attack_type = random.choices(physical_attack_list, weights)[0] # △重み（確率）を使いたい場合はchoices。返り値がリストなので [0] をつける

        if physical_attack_type == 1:
            damage = self.attack * 2
            damage_text = f"{self.name}の攻撃。会心の一撃！{attack_target.name}に{damage}のダメージ。"

        elif physical_attack_type == 2:
            damage = 0
            damage_text = f"{self.name}の攻撃。ミス！{attack_target.name}は攻撃を受けない。"

        else:
            damage = self.attack
            damage_text = f"{self.name}の攻撃。{attack_target.name}に{damage}のダメージ。"

        attack_target.hp -= damage
        if attack_target.hp <= 0 :
            attack_target.hp = 0
            return f"{damage_text}{attack_target.name}のHPは{attack_target.hp}。{attack_target.name}{attack_message}"
        else:
            return f"{damage_text}{attack_target.name}のHPは{attack_target.hp}。"
        # モンスター・プレイヤー共通、物理攻撃おわり



    # 全員、攻撃魔法（単体攻撃 も全体攻撃も）
    def common_magic(self, targets, attack_message, required_MP, default, magic, target_type):
        if self.mp < required_MP:
            return f"{self.name}は{magic}を唱えた。MPが足りない！(現在MP:{self.mp})"
        else:
            self.mp -= required_MP
            damage = self.magic * default
            magic_text = f"{self.name}は{magic}を唱えた。\n"

            if target_type == "single" :
                target_list = [random.choice(targets)]
            else:
                target_list = targets
            # 略してtarget_list = [random.choice(targets)] if target_type == "single" else targets

            for target in target_list:
                target.hp -= damage

                if target.hp <= 0:
                    target.hp = 0
                    magic_text += f"{target.name}に{damage}のダメージ。{target.name}のHPは{target.hp}。{target.name}{attack_message}\n"
                else:
                    magic_text += f"{target.name}に{damage}のダメージ。{target.name}のHPは{target.hp}。\n"

            return magic_text
            # モンスター・プレイヤー共通、攻撃魔法おわり



    # 全員、回復魔法
    def common_hoimi(self, heal_targets, attack_message, required_MP, default, magic):
        if self.mp < required_MP:
            return f"{self.name}は{magic}を唱えた。MPが足りない！(現在MP:{self.mp})"
        else:
            self.mp -= required_MP
            heal = self.magic * default
            heal_text =  f"{self.name}は{magic}を唱えた。\n"

            for heal_target in heal_targets:
                heal_target.hp += heal

                if heal_target.hp > heal_target.max_hp:
                    heal_target.hp = heal_target.max_hp

                heal_text +=  f"{heal_target.name}のHPは{heal_target.hp}に回復した。\n"
            return heal_text
            # モンスター・プレイヤー共通、回復魔法おわり



# モンスター共通の攻撃を設定
# △モンスタークラスの作成（Characterを継承しているので、Character 定義された共通のメソッドも使える）
class Monster(Character):
    # ①-2モンスターの攻撃（ランダム）と生存判定(各攻撃後に生存者を確認し、どちらかが全滅していれば戦闘を終了)
                        # 　敵　 　味方
    def attack_start(self, enemys, allys, attack_message, players, monsters):
        result_texts = []
        random.shuffle(allys)
        for ally in allys:
            result_texts.append(f"\n☆{ally.name}の番☆")
            result_texts.append(ally.select_action_type(enemys, allys, attack_message))


            enemys[:] = [enemy for enemy in enemys if enemy.hp > 0]
                # △[:]をつけることによって、下記のmonstersリスト or playersリストを更新してくれる
            allys[:]  = [ally for ally in allys if ally.hp > 0]

            if not monsters:
                result_texts.append("\n！！敵をすべて倒した！！")
                return True, result_texts
            elif not players:
                result_texts.append("\n～ゲームオーバー～")
                return True, result_texts
        return None, result_texts


    # ②-2攻撃方法（物理攻撃か他の攻撃か）をランダムに決定（モンスター用）
    def select_action_type(self, attack_target, heal_target, attack_message):
        action_type = random.randint(1,2)
        if action_type == 1:
            return self.select_attack_target(attack_target, heal_target, attack_message)
        else:
            return self.attack_type2(attack_target, heal_target, attack_message)


    # ③-1物理攻撃ターゲットをランダムに決定（モンスター用）
    def select_attack_target(self, attack_targets, heal_targets, attack_message):
        # 生きているプレイヤーをリストに入れる
        alive_list = [attack_target for attack_target in attack_targets if attack_target.hp > 0]
        attack_target = random.choice(alive_list)
        return self.common_attack(attack_target, heal_targets, attack_message)


    # ⑤-1回復する味方モンスターランダムに決定（モンスター用）
    def select_heal_target(self, heal_targets, attack_message,
                                  required_MP, default, magic, target_type):
        # 生きている仲間をリストに入れる
        alive_list = [heal_target for heal_target in heal_targets if heal_target.hp > 0]
        if target_type == "single" :
            heal_targets = [random.choice(alive_list)]
        else:
            heal_targets = alive_list
        return self.common_hoimi(heal_targets, attack_message, required_MP, default, magic)
# モンスター共通の攻撃おわり


# 各モンスター固有の攻撃方法を定義
# モンスター1　メイジドラキー
class Doraki(Monster):
    # 単体に「ヒャド」を唱える
    def attack_type2(self, attack_target, heal_target, attack_message):
        return self.common_magic(attack_target, attack_message, 3, 3, "ヒャド", "single")

# モンスター2　キメラ
class Kimera(Monster):
    # 全体に「ギラ」を唱える
    def attack_type2(self, attack_target, heal_target, attack_message):
        return self.common_magic(attack_target, attack_message, 4, 5, "ギラ", "all")

# モンスター3　ホイミスライム
class h_suraimu(Monster):
    # 単体に「ホイミ」を唱える
    def attack_type2(self, attack_target, heal_target, attack_message):
        return self.select_heal_target(heal_target, attack_message,3, 6, "ホイミ", "single")
# モンスター達の攻撃処理おわり




# プレイヤー共通の攻撃を設定
# △プレイヤークラスの作成
class Player(Character):
    # ①-1プレイヤーの攻撃と生存判定(各攻撃後に生存者を確認し、どちらかが全滅していれば戦闘を終了)
                       # 　 敵　 　味方
    def attack_start(self, enemys, allys, attack_message, players, monsters):
        result_texts = []  #
        result_texts.append(self.select_action_type(enemys, allys, attack_message))

        enemys[:] = [monster for monster in monsters if monster.hp > 0] #
            # △[:]をつけることによって、下記のmonstersリスト or playersリストを更新してくれる
        allys[:]   = [player for player in players if player.hp > 0]

        if not enemys:
            result_texts.append("\n！！敵をすべて倒した！！")
            return True, result_texts
        elif not allys:
            result_texts.append("\n～ゲームオーバー～")
            return True, result_texts
        return None, result_texts


    # ②-1攻撃方法（物理攻撃か他の攻撃か）を入力して決定（プレイヤー用）
    def select_action_type(self, attack_target, heal_target, attack_message):
        print("攻撃方法を選んでください：")

        while True: # 正しい番号が入力されたらそのターゲットを選択して break（ループを抜ける）
            try:
                action_type = int(input("1：攻撃、2：魔法　選択→"))
                if action_type == 1:
                    return self.select_attack_target(attack_target, heal_target, attack_message)
                                                   # return がループを抜けるからbreakはいらない
                elif action_type == 2:
                    return self.attack_type2(attack_target, heal_target, attack_message)
                else: # それ以外の数字を選択時
                         print("範囲内の半角数字を入力してください。")
            except ValueError: # 半角数字ではない場合
                print("半角数字で入力してください。")


    # ③-1物理攻撃ターゲットをインプット入力して決定（プレイヤー用）
    def select_attack_target(self, attack_targets, heal_targets, attack_message):
        # 生きているキャラをリストに入れる
        alive_list = [attack_target for attack_target in attack_targets if attack_target.hp > 0]

        # 攻撃するモンスターを選択
        print("攻撃するモンスターを選んでください：")
        for i, attack_target in enumerate(alive_list, 1): # ※enumerateはリストの各要素に順番の番号をつけてくれる関数。番号を1からつける
                                                          # ※enumerate() は2つの値（番号とターゲット） を返す
            print(f"{i}: {attack_target.name}")

        while True: # 範囲内の番号が入力されたらループを抜けてreturn
            try:
                choice = int(input("半角数字を入力→ "))
                if 1 <= choice <= len(alive_list): # 入力された番号が リストの範囲内かチェック（※1以上 かつ len(alive_list) 以下）
                    attack_target = alive_list[choice - 1] # ※リストは 0番目から始まる ので、入力した番号から1を引く
                    break

                else: # ※それ以外の数字を選択時
                    print("範囲内の半角数字を入力してください。")
            except ValueError: # ※半角数字ではない場合
                print("半角数字で入力してください。")
        return self.common_attack(attack_target, heal_targets, attack_message)


    # ⑤-1回復の処理（プレイヤー用）
    def select_heal_target(self, attack_target, heal_targets, attack_message,
                                  required_MP, default, magic, target_type):
        # 生きていて、最大HPではない味方だけを抽出をリストに入れる
        alive_list = [heal_target for heal_target in heal_targets if heal_target.hp > 0 and heal_target.hp < heal_target.max_hp]
        if not alive_list: # リストが空ならもう一度攻撃方法選択
            return self.select_action_type(attack_target, heal_targets, attack_message)

        if target_type == "single" : # 単体回復（ホイミ）の時は、回復相手をインプット入力して決定
            print("回復するプレイヤーを選んでください：")
            for i, heal_target in enumerate(alive_list, 1): # ※enumerateについて↑
                print(f"{i}: {heal_target.name} (HP:{heal_target.hp}/{heal_target.max_hp})")

            while True: # ※範囲内の番号が入力されたらループを抜けてreturn
                try:
                    choice = int(input("半角数字を入力→ "))
                    if 1 <= choice <= len(alive_list):
                        heal_targets = [alive_list[choice - 1]]
                        break

                    else: # ※それ以外の数字を選択時
                        print("範囲内の半角数字を入力してください。")
                except ValueError: # ※半角数字ではない場合
                     print("半角数字で入力してください。")
            return self.common_hoimi(heal_targets, attack_message, required_MP, default, magic)

        else:
            heal_targets = alive_list # 全体回復（ベホマラー）なら、生きている味方全員をリストにいれる
        return self.common_hoimi(heal_targets, attack_message, required_MP, default, magic)
# プレイヤー共通の攻撃おわり



# 各プレイヤーの攻撃方法　3人
# 1.騎士
class Knight(Player):
    def attack_type2(self, attack_target, heal_target, attack_message):
        # 単体に「ホイミ」を唱える魔法回復
        return self.select_heal_target(attack_target, heal_target, attack_message,3, 6, "ホイミ", "single")

# 2.fire魔法使い（攻撃）
class WizardA(Player):
    # 魔法攻撃(メラとイオ)か決める
    def attack_type2(self, attack_target, heal_target, attack_message):
        magic_types = int(input("1：メラ（MP2）、2：イオ（MP5）　選択→"))
        # 単体に「メラ」を唱える魔法攻撃
        if magic_types == 1:
            return self.common_magic(attack_target, attack_message, 2, 2, "メラ", "single")
        # 全体に「イオ」を唱える魔法攻撃
        elif magic_types == 2:
            return self.common_magic(attack_target, attack_message, 5, 4, "イオ", "all")


# 3.holy魔法使い（回復）
class WizardB(Player):
    # 回復魔法(ホイミかベホマラー)を決める
    def attack_type2(self, attack_target, heal_target, attack_message):
        magic_types = int(input("1：ホイミ（MP3）、2：ベホマラー（MP10）　選択→"))
        #magic_types = random.randint(1,2)
        # 単体に「ホイミ」を唱える魔法回復
        if magic_types == 1:
            return self.select_heal_target(attack_target, heal_target, attack_message,3, 6, "ホイミ", "single")
        # 全体に「ベホマラー」を唱える魔法回復を定義
        elif magic_types == 2:
            return self.select_heal_target(attack_target, heal_target, attack_message, 10, 6, "ベホマラー","all")
      # 魔法使い（回復）の攻撃処理おわり


# 戦闘結果のメッセージをまとめて表示するための関数
def show_battle_log(result_texts):
    for text in result_texts:
        print(text)


# モンスターの初期データ（名前・HP・MPなど）を設定
m_dorakiA    = Doraki ("メイジドラキーA",30, 16, 19, 4)
m_dorakiB    = Doraki ("メイジドラキーB",30, 16, 19, 4)
kimeraA    = Kimera   ("キメラA",        50, 23, 14, 3)
kimeraB    = Kimera   ("キメラB",        50, 23, 14, 3)
h_suraimuA = h_suraimu("ホイミスライムA",40, 14, 30, 5)
h_suraimuB = h_suraimu("ホイミスライムB",40, 14, 30, 5)

# プレイヤーの情報
knight   = Knight ("騎士",        150, 15, 30, 5)
wizardA  = WizardA("fire魔法使い",130,  8, 21, 5)
wizardB  = WizardB("holy魔法使い",110,  6, 39, 5)

# ループ処理のためにモンスターとプレイヤーのリストを作成
monsters = [kimeraA, kimeraB, h_suraimuA, h_suraimuB, m_dorakiA, m_dorakiB]
players  = [knight, wizardA, wizardB]

# ---------------------------------------------------------

# モンスターの出現情報
for monster in monsters:
    print(f"{monster.name}が現れた。")

turn = 0 # ※ターン数表示のため
while players and monsters:   # リストにプレイヤーもモンスターもいる時は繰り返す（※monstersと書くだけで誰かがいることを指す）
    turn += 1
    print(f"\n●{turn}ターン目●")

    # プレイヤー全員の攻撃
    players[:]  = [player for player in players if player.hp > 0] # プレイヤー生存リスト
    random.shuffle(players)  # 攻撃する順番をランダムにする
    for player in players:   # 順番に攻撃
        print(f"\n☆{player.name}の番☆")
        attack_result, result_texts = player.attack_start(monsters, players, "を倒した。", players, monsters)
                # attack_startの生存判定と集めたテキストが返る
        show_battle_log(result_texts)  # result_texts の表示
        if attack_result:  # attack_startでTrueなら終わる
            break


    # モンスター全員の攻撃
    if monsters:  # 空でない場合のみ攻撃
        attack_result, result_texts = monsters[0].attack_start(players, monsters, "は気絶した＞＜。。", players, monsters)
        show_battle_log(result_texts)    # attack_resultはTrueかNone、 logsはresult_texts　が返る
        if attack_result:
            break



    # ターン終了時に生きているキャラクターの表示
    print("\n--- 現在のステータス ---")
    print("プレイヤー→", ",    ".join([f"{player.name}のHP:{player.hp}・MP:{player.mp}" for player in players]))
    print("モンスター→", ",    ".join([f"{monster.name}" for  monster in monsters]))
    print("-----------------------")



キメラAが現れた。
キメラBが現れた。
ホイミスライムAが現れた。
ホイミスライムBが現れた。
メイジドラキーAが現れた。
メイジドラキーBが現れた。

●1ターン目●

☆holy魔法使いの番☆
攻撃方法を選んでください：
1：攻撃、2：魔法　選択→1
攻撃するモンスターを選んでください：
1: キメラA
2: キメラB
3: ホイミスライムA
4: ホイミスライムB
5: メイジドラキーA
6: メイジドラキーB
半角数字を入力→ 5
holy魔法使いの攻撃。メイジドラキーAに6のダメージ。メイジドラキーAのHPは24。

☆fire魔法使いの番☆
攻撃方法を選んでください：
1：攻撃、2：魔法　選択→2
1：メラ（MP2）、2：イオ（MP5）　選択→2
fire魔法使いはイオを唱えた。
キメラAに20のダメージ。キメラAのHPは30。
キメラBに20のダメージ。キメラBのHPは30。
ホイミスライムAに20のダメージ。ホイミスライムAのHPは20。
ホイミスライムBに20のダメージ。ホイミスライムBのHPは20。
メイジドラキーAに20のダメージ。メイジドラキーAのHPは4。
メイジドラキーBに20のダメージ。メイジドラキーBのHPは10。


☆騎士の番☆
攻撃方法を選んでください：
1：攻撃、2：魔法　選択→1
攻撃するモンスターを選んでください：
1: キメラA
2: キメラB
3: ホイミスライムA
4: ホイミスライムB
5: メイジドラキーA
6: メイジドラキーB
半角数字を入力→ 5
騎士の攻撃。メイジドラキーAに15のダメージ。メイジドラキーAのHPは0。メイジドラキーAを倒した。

☆ホイミスライムAの番☆
ホイミスライムAの攻撃。fire魔法使いに14のダメージ。fire魔法使いのHPは116。

☆ホイミスライムBの番☆
ホイミスライムBはホイミを唱えた。
メイジドラキーBのHPは30に回復した。


☆キメラBの番☆
キメラBはギラを唱えた。
holy魔法使いに15のダメージ。holy魔法使いのHPは95。
fire魔法使いに15のダメージ。fire魔法使いのHPは101。
騎士に15のダメージ。騎士のHPは135。


☆メイジドラキーBの番☆
メイジドラキーBの攻撃。holy魔法使いに16のダメージ。holy魔法使いのHPは79。

In [ ]:
# 16dorakue_ver5

# attack_result, result_texts = players.attack_start 　　 →①-1へ
 #           (1monsters,       2player,       3"を倒した。",          players, monsters)
# attack_result, result_texts = monsters[0].attack_start　→①-2へ
 #           (1players,        2monsters,      3"は気絶した＞＜。。", players, monsters)

# Character（親クラス：共通機能）
# │　 ③-3物理攻撃：common_attack
# │　 ④魔法攻撃：common_magic(単体も全体も)
# │　 ⑤-2回復魔法：common_hoimi(単体も全体も)
# │
# ├──Monster（モンスター共通）
# │     │ ①-2モンスターの攻撃と生存判定（ランダム）：attack_start　→　②-2へ
# │     │
# │     │ ②-2攻撃方法選択（物理攻撃か他攻撃か決める）（ランダム）：select_action_type
# │     │　※物理攻撃だった時は各③-2へ：select_attack_target
# │     │　※それ以外は各モンスターへ：attack_type2
# │     │
# │     │ ③-2物理攻撃ターゲットを決める（ランダム）:select_attack_target  　→③-3へ
# │ 　  │
# │     │ ⑤-1回復ターゲットを決める（ランダム）:select_heal_target  →　⑤-2へ
# │     │
# │　　 ├──メイジドラキー
# │　 　│　 　└──攻撃魔法（単体）ヒャド　→　④へ
# │　 　│
# │　　 ├──キメラ
# │　 　│　 　└──攻撃魔法（全体）ギラ　→　④へ
# │　 　│
# │　　 └──ホイミスライム
# │　 　  　 　└──回復魔法（ホイミ）　→　Monster⑤-1へ
# │
# └──Player（プレイヤー共通）
#        │ ①-1プレイヤーの攻撃と生存判定（入力）：attack_start　→　②-1へ
#        │
# 　　　 │ ②-1攻撃方法選択（物理攻撃か他攻撃か決める）（入力）：select_action_type
# 　　　 │　※物理攻撃だった時は各③-1へ：select_attack_target
# 　　　 │　※それ以外は各プレイヤーへ：attack_type2
#        │
#        │ ③-1物理攻撃ターゲットを決める（入力）:select_attack_target 　→③-3へ
#        │
#        │ ⑤-1回復ターゲットを決める（入力）:select_heal_target →　⑤-2へ
# 　　　 │
#   　　 ├──騎士
# 　　　 │　 　└──回復魔法（ホイミ）　→　Player⑤-1へ
# 　　　 │
#   　　 ├──fire魔法使い（攻撃）
# 　　　 │　 　└──攻撃魔法
# 　　 　│　　        ├──攻撃魔法（単体）メラ　→　④へ
# 　　 　│ 　　       └──攻撃魔法（全体）イオ　→　④へ
# 　　 　│
# 　　   └──holy魔法使い（回復）
# 　　　  　  　└──回復魔法
# 　　　  　  　       ├──回復魔法（ホイミ）　　→　Player⑤-1へ
# 　　　  　  　       └──回復魔法（ベホマラー）→　Player⑤-1へ
